### Seguridad Ecopetrol

## Contenido
* Importación librerías
* Carga información
* Calculo General
* Series de tiempo
 * ITEM
 * DES_ACCION
* Preciciones
* Revisión Metricas
* Seleccion Mejor Método
* Predicciones futuras

## Importación Librerías

In [1]:
import pandas as pd
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import datetime, date, timedelta
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt,ExponentialSmoothing
import numpy as np
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import warnings
warnings.filterwarnings('ignore')

## Carga información

In [2]:
path_data = ''
file= f'../Bases_Datalake/VW_TRAF_SEGURIDAD_SEMANA.csv'

df = pd.read_csv(f'{path_data}{file}', sep='|', decimal='.')

FileNotFoundError: [Errno 2] No such file or directory: '../Bases_Datalake/VW_TRAF_SEGURIDAD_SEMANA.csv'

In [ ]:
df.head()

In [ ]:
len(df)

##  Cálculo General

In [ ]:
dfTotalV2=df.groupby(['FEC_MUESTRA'])[['VAL_CANTIDAD']].agg('sum')
dfTotalV2.index=pd.to_datetime(dfTotalV2.index)

In [ ]:
dfTotalV2

In [ ]:
#dfTotalV2.asfreq(freq='W-WED').plot()

In [ ]:
#dfTotalV2.asfreq(freq='W-WED',method='bfill').plot()

In [ ]:
#dfTotalV2.asfreq(freq='W-WED').interpolate().plot()
dfTotalV2=dfTotalV2.asfreq(freq='W-WED').interpolate()

### Seleccion Fecha Train y Test

In [ ]:
fechaMaxima=max(dfTotalV2.index)
fechaMaxima

In [ ]:
fechaMaxTrain=dfTotalV2.index[-5]
fechaMinTest=dfTotalV2.index[-4]

In [ ]:
df_serie_train=dfTotalV2[:fechaMaxTrain]
df_serie_test=dfTotalV2[fechaMinTest:]

In [ ]:
df_serie_test

In [ ]:
ind = pd.date_range(df_serie_test.index.min(), periods = 4, freq ='W-WED', name='FEC_MUESTRA')

In [ ]:
ind

In [ ]:
modelAutoARIMA=auto_arima(df_serie_train['VAL_CANTIDAD'], max_p = 4, max_q = 4,trace=False)
P_ModeloAUtoArima=pd.DataFrame(modelAutoARIMA.predict(n_periods = 4), index=ind)

In [ ]:
dfTotalV2['p_ARIMA']=P_ModeloAUtoArima

In [ ]:
## Exponential smooth
modelSM = SimpleExpSmoothing(df_serie_train['VAL_CANTIDAD'])
modelSM._index=df_serie_train.index

In [ ]:
df_serie_train['VAL_CANTIDAD']

In [ ]:
#P_ModeloAUtoArima
P_modelSM=modelSM.fit(smoothing_level=.5).forecast(5)
P_modelSM

In [ ]:
dfTotalV2['p_ES']=P_modelSM

In [ ]:
## Holt

In [ ]:
modelHl = Holt(df_serie_train['VAL_CANTIDAD'])
modelHl._index=df_serie_train.index

In [ ]:
P_modelHl=modelHl.fit(optimized=True).forecast(5)
P_modelHl

In [ ]:
dfTotalV2['p_HL']=P_modelHl

In [ ]:
modelESD=ExponentialSmoothing(df_serie_train['VAL_CANTIDAD'], trend='mul', seasonal=None, damped=True)
P_modelESD=modelESD.fit().forecast(5)

In [ ]:
P_modelESD

In [ ]:
dfTotalV2['p_ESD']=P_modelESD

In [ ]:
#dfTotalV2.reset_index().plot('FEC_MUESTRA',['VAL_CANTIDAD','p_ARIMA','p_ES','p_HL','p_ESD'], figsize=(15,8))

## 2.  Des_Evento

In [ ]:
dfDesEvento=df.groupby(['FEC_MUESTRA','DES_EVENTO'])['VAL_CANTIDAD'].agg('sum').reset_index('DES_EVENTO')

In [ ]:
dfDesEvento

In [ ]:
## agregar ITEM Malware y DES Accion Bloqueo
df_Accion=df[df['DES_ACCION']=='Bloqueos'].groupby(['FEC_MUESTRA'])[['VAL_CANTIDAD']].agg('sum')
df_Accion.columns=['ACCION_Bloqueos']

In [ ]:
df_Accion

In [ ]:
df_Item=df[df['ITEM']=='Malware'].groupby(['FEC_MUESTRA'])[['VAL_CANTIDAD']].agg('sum')
df_Item.columns=['ITEM_Malware']

In [ ]:
df_Item

In [ ]:
dfDesEvento_T=pd.pivot_table(dfDesEvento.reset_index(), index='FEC_MUESTRA', columns='DES_EVENTO', values='VAL_CANTIDAD')

In [ ]:
dfDesEvento_T['ACCION_Bloqueos']=df_Accion['ACCION_Bloqueos']
dfDesEvento_T['ITEM_Malware']=df_Item['ITEM_Malware']

In [ ]:
dfDesEvento_T

In [ ]:
dfDesEvento_T.index=pd.to_datetime(dfDesEvento_T.index)

In [ ]:
dfDesEvento_T=dfDesEvento_T.asfreq(freq='W-WED').interpolate()
dfDesEvento_T.fillna(method="bfill",inplace=True)

In [ ]:
dfDesEvento_T

In [ ]:
de_serie_train=dfDesEvento_T[:fechaMaxTrain]
de_serie_test=dfDesEvento_T[fechaMinTest:]
grupos=dfDesEvento_T.columns

In [ ]:
grupos

In [ ]:
de_serie_train

In [ ]:
prefix=['P_ARIMA_','p_ES_','p_HL_','p_ESD_','p_ETS_']

In [ ]:
M_error={}
for col in grupos: 
    M={}
    #model AutoARIMA
    modelAutoARIMA=auto_arima(de_serie_train[col],trace=False)
    modelAutoARIMA.fit(de_serie_train[col])
    P_ModeloAUtoArima=pd.DataFrame(modelAutoARIMA.predict(n_periods = 4), index=ind)
    dfDesEvento_T['P_ARIMA_'+col]=P_ModeloAUtoArima
    # model Exponential Smooting
    modelSM = SimpleExpSmoothing(de_serie_train[col])
    #modelSM._index=df_serie_train.index
    P_modelSM=modelSM.fit(smoothing_level=.5).forecast(4)
    dfDesEvento_T['p_ES_'+col]=P_modelSM
    #Holt
    modelHl = Holt(de_serie_train[col])
    P_modelHl=modelHl.fit(optimized=True).forecast(4)    
    dfDesEvento_T['p_HL_'+col]=P_modelHl
    #ES tend
    modelESD=ExponentialSmoothing(de_serie_train[col], trend='mul', seasonal=None, damped=True)
    P_modelESD=modelESD.fit().forecast(4)
    dfDesEvento_T['p_ESD_'+col]=P_modelESD    
    ##ETS
    modelETS = ETSModel(de_serie_train[col],seasonal_periods=4,error="add",trend="add",seasonal="add", damped_trend=True,)
    modelETS = modelETS.fit()
    start=len(de_serie_train[col])
    P_modelETS=modelETS.get_prediction(start=start, end=start+3).summary_frame(alpha=0.05)['mean']
    dfDesEvento_T['p_ETS_'+col] = P_modelETS
                
    ## calculo de errores
    M['RMSE_ARIMA']=mean_squared_error(de_serie_test[col],P_ModeloAUtoArima, squared=False)
    M['MAPE_ARIMA']=mean_absolute_percentage_error(de_serie_test[col],P_ModeloAUtoArima)
    M['RMSE_ES']=mean_squared_error(de_serie_test[col],P_modelSM, squared=False)
    M['MAPE_ES']=mean_absolute_percentage_error(de_serie_test[col],P_modelSM)
    M['RMSE_HL']=mean_squared_error(de_serie_test[col],P_modelHl, squared=False)
    M['MAPE_HL']=mean_absolute_percentage_error(de_serie_test[col],P_modelHl)
    M['RMSE_ESD']=mean_squared_error(de_serie_test[col],P_modelESD, squared=False)
    M['MAPE_ESD']=mean_absolute_percentage_error(de_serie_test[col],P_modelESD)
    M['RMSE_ETS']=mean_squared_error(de_serie_test[col],P_modelETS, squared=False)
    M['MAPE_ETS']=mean_absolute_percentage_error(de_serie_test[col],P_modelETS)
    
    
    M_error[col]=M

In [ ]:
dfDesEvento_T

In [ ]:
M_error

In [ ]:
df_error=pd.DataFrame.from_dict(M_error, orient='index')
df_error

In [ ]:
df_error=df_error[df_error.columns.sort_values()]

In [ ]:
df_error

##  Seleccionar modelo por serie

In [ ]:
colError=df_error.filter(regex='^MAPE',axis=1).columns.values

In [ ]:
colMetodos={x.replace('MAPE_','') for x in colError}

In [ ]:
colMetodos

In [ ]:
minError=list(df_error.filter(regex='^MAPE',axis=1).min(axis=1).values)

In [ ]:
minError

In [ ]:
modeloSeleccionado=[np.nan]*len(df_error)
i=0
for ind,row in df_error.filter(regex='^MAPE',axis=1).iterrows():
    auxRow=list(row.values)
    auxInd=auxRow.index(minError[i])
    modeloSeleccionado[i]=colError[auxInd].replace('MAPE_','')
    print(auxRow)
    i=i+1

In [ ]:
modeloSeleccionado

In [ ]:
df_error['seleccion']=modeloSeleccionado

In [ ]:
df_error

### Visual resultados

In [ ]:
"""
fig, axs = plt.subplots(3, 3 ,figsize=(15,10))
i=0
for col in grupos:
    dfDesEvento_T[[col]].plot(ax=axs[i//3,i%3])    
    axs[i//3,i%3].title.set_text(col)
    for model in prefix:
        dfDesEvento_T[[model+col]].plot(ax=axs[i//3,i%3])
    axs[i//3,i%3].legend(prop={'size':10})
    i=i+1
fig.tight_layout()"""

##  Pronosticos futuros

In [ ]:
ind_P = pd.date_range(fechaMaxima, periods = 5, freq ='W-WED', name='FEC_MUESTRA')[1:]

In [ ]:
ind_P

In [ ]:
dfResultado=pd.DataFrame(columns=colMetodos, index=ind_P)

In [ ]:
dfResultado['SERIE']=np.nan

In [ ]:
#dfResultado=dfResultado.reset_index()

In [ ]:
dfResultado

In [ ]:
dfResultadoFinal=dfResultado.copy()
for col in grupos:     
    #model AutoARIMA
    modelAutoARIMA=auto_arima(dfDesEvento_T[col],trace=False)
    modelAutoARIMA.fit(dfDesEvento_T[col])
    tf=pd.DataFrame(modelAutoARIMA.predict(n_periods = 4), index=ind_P, columns=['ARIMA'])
    modelSM = SimpleExpSmoothing(dfDesEvento_T[col])    
    P_modelSM=modelSM.fit(smoothing_level=.5).forecast(4)
    tf['ES']=P_modelSM
    #Holt
    modelHl = Holt(dfDesEvento_T[col])
    P_modelHl=modelHl.fit(optimized=True).forecast(4)    
    tf['HL']=P_modelHl    
    #ES tend
    modelESD=ExponentialSmoothing(dfDesEvento_T[col], trend='mul', seasonal=None, damped=True)
    P_modelESD=modelESD.fit().forecast(4)
    tf['ESD']=P_modelESD 
    #ETS
    modelETS = ETSModel(dfDesEvento_T[col],seasonal_periods=4,error="add",trend="add",seasonal="add", damped_trend=True,)
    modelETS = modelETS.fit()
    start=len(dfDesEvento_T[col])
    P_modelETS=modelETS.get_prediction(start=start, end=start+3).summary_frame(alpha=0.05)['mean']
    tf['ETS']=P_modelETS 
    #dfDesEvento_T['p_ETS_'+col] = P_modelETS
    
    #serie
    tf['SERIE']=col
    dfResultadoFinal=dfResultadoFinal.append(tf)
    

In [ ]:
dfResultadoFinal=dfResultadoFinal[dfResultadoFinal['SERIE'].notnull()]

In [ ]:
dfResultadoFinal.reset_index(inplace=True)

In [ ]:
dfResultadoFinal

In [ ]:
# seleccionar mejor resultado
dfResultadoFinal['VAL_CANTIDAD']=0
dfResultadoFinal['metodo']=np.nan
for ind,row in dfResultadoFinal.iterrows():
    #print(ind)
    met=df_error.loc[row['SERIE'],'seleccion']
    #print(met)
    dfResultadoFinal.loc[ind,'metodo']=met
    dfResultadoFinal.loc[ind,'VAL_CANTIDAD']=dfResultadoFinal.loc[ind,met]
    #dfResultadoFinal

In [ ]:
#Eliminar predicciones negativas
dfResultadoFinal['VAL_CANTIDAD']=np.where(dfResultadoFinal['VAL_CANTIDAD']<0,0,dfResultadoFinal['VAL_CANTIDAD'])

In [ ]:
dfResultadoFinal

In [ ]:
formato_envio_pred=dfResultadoFinal[['FEC_MUESTRA','SERIE','VAL_CANTIDAD']]
formato_envio_pred['tipo']='pred'

In [ ]:
formato_envio_pred

### Formato envío

In [ ]:
grupos

In [ ]:
dfDesEvento_T[grupos]

In [ ]:
formato_envio_hist=dfDesEvento_T[grupos].reset_index().melt(id_vars=['FEC_MUESTRA'],var_name='SERIE', value_name='VAL_CANTIDAD')

In [ ]:
formato_envio_hist['tipo']='hist'

In [ ]:
formato_envio_hist

In [ ]:
formato_envio=formato_envio_hist.append(formato_envio_pred)

In [ ]:
formato_envio['VAL_CANTIDAD']=formato_envio['VAL_CANTIDAD'].round(2)

In [ ]:
formato_envio

In [ ]:
fechaMinPron=formato_envio_pred['FEC_MUESTRA'].min()
fechaMaxPron=formato_envio_pred['FEC_MUESTRA'].max()
#nomFile='Eco_seg_'+fechaMinPron.strftime("%Y%m%d")+'_'+fechaMaxPron.strftime("%Y%m%d")
nomFile='../Bases_Post_Transformaciones/Eco_seg_'

In [ ]:
for serie in grupos:
    print(serie)
    formato_envio[formato_envio['SERIE']==serie].to_csv(nomFile+'_'+serie+'.csv',index=False)

In [ ]:
#formato_envio.to_csv(nomFile+'.csv',index=False)